<a href="https://colab.research.google.com/github/AmalTJohnson/Learn-to-Code-21/blob/main/Daily_load_predict_prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt




In [ ]:
df = pd.read_csv('/content/Training_per_day.csv',index_col='hour',parse_dates=True)
df.index.freq='MS'

In [ ]:
df.head

In [ ]:
df.plot(figsize=(12,6))

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
len(df)

In [ ]:
train = df.iloc[:20]
test = df.iloc[20:]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()



In [ ]:
df.head(),df.tail()

In [ ]:
scaler.fit(train)
scaled_train = scaler.transform(train)
scaled_test = scaler.transform(test)

In [ ]:
scaled_train[1:]

In [ ]:
from keras.preprocessing.sequence import TimeseriesGenerator

In [ ]:
# define generator
n_input = 3
n_features = 1
generator = TimeseriesGenerator(scaled_train, scaled_train, length=n_input, batch_size=1)


In [ ]:
print(len(scaled_train),len(generator))

In [ ]:
inp,res = generator[0]
print(f'Given the Array: \n{inp.flatten()}')
print(f'Predict this res: \n {res}')


In [ ]:
inp.shape


In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [ ]:
model = Sequential()
model.add(LSTM(20,activation='relu',input_shape=(n_input,n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.summary()

In [ ]:
model.fit(generator,epochs=50)

In [ ]:
loss_per_epoch = model.history.history['loss']
plt.plot(range(len(loss_per_epoch)),loss_per_epoch)

In [ ]:
last_vals = scaled_train[:3]

In [ ]:
# @title
last_vals = last_vals.reshape((1, n_input, n_features))

In [ ]:
model.predict(last_vals)


In [ ]:
scaled_test[0]


In [ ]:
test_predictions = []

first_eval_batch = scaled_train[-n_input:]
current_batch = first_eval_batch.reshape((1, n_input, n_features))

for i in range(len(test)):

    # get the prediction value for the first batch
    current_pred = model.predict(current_batch)[0]

    # append the prediction into the array
    test_predictions.append(current_pred)

    # use the prediction to update the batch and remove the first value
    current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

In [ ]:
test_predictions


In [ ]:
test.head()

In [ ]:
true_predictions = scaler.inverse_transform(test_predictions)

In [ ]:
test['Predictions'] = true_predictions


In [ ]:
test.plot(figsize=(14,5))

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
rmse=sqrt(mean_squared_error(test['load'],test['Predictions']))
print(rmse)
